In [1]:
import logging;
logging.disable(logging.WARNING);

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
!conda install -y gdown

In [ ]:
# https://drive.google.com/file/d/1-649JNvw15TYDciHpMlJKXQ08_2lKJF4/view
!gdown --id "1-649JNvw15TYDciHpMlJKXQ08_2lKJF4"

In [ ]:
# https://drive.google.com/file/d/1-8-Pv2lowjIAalgaY4gUMdYhVadPp0Gn/view?usp=sharing
!gdown --id "1-8-Pv2lowjIAalgaY4gUMdYhVadPp0Gn"

In [ ]:
# https://drive.google.com/file/d/1m11t37h2bLU2wmd41aTEQREGWsFJGyfj/view?usp=sharing
!gdown --id "1m11t37h2bLU2wmd41aTEQREGWsFJGyfj"

In [ ]:
# https://drive.google.com/file/d/1--PbeWSlq26rId939CNjGBWQV8Dcg6p5/view?usp=sharing
!gdown --id "1--PbeWSlq26rId939CNjGBWQV8Dcg6p5"

In [ ]:
# https://drive.google.com/file/d/1dHb0OWltqjqf7YvavHOrw45GFpQiHXIg/view?usp=sharing
!gdown --id "1dHb0OWltqjqf7YvavHOrw45GFpQiHXIg"

In [ ]:
# https://drive.google.com/file/d/1-0nKSznYVRZsdd84IEUPWF35_aVURsB_/view?usp=sharing
!gdown --id "1-0nKSznYVRZsdd84IEUPWF35_aVURsB_"

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration;
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer;

In [ ]:
import torch;
import torch.nn as nn;
import torch.optim as optim;
import torch.nn.functional as F;
from torch.utils.data import Dataset, DataLoader;

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
print(device);

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base');
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base');

In [ ]:
class CustomDataset(nn.Module):
    def __init__(self,path_x,path_tags,path_y,tokenizer,max_seq_length,max_target_length):
        super().__init__();

        self.max_seq_length = max_seq_length;
        self.max_target_length = max_target_length;

        with open(path_x,'r') as f:
            self.x = f.readlines();
        with open(path_y,'r') as f:
            self.y = f.readlines();
        with open(path_tags,'r') as f:
            self.tags = f.readlines();

    def __len__(self):
        return len(self.x);

    def __getitem__(self,idx):
        x = tokenizer(self.tags[idx],self.x[idx],max_length=self.max_seq_length,truncation=True,padding='max_length',return_tensors='pt');
        y = tokenizer(self.y[idx],max_length=self.max_target_length,truncation=True,padding='max_length',return_tensors='pt');
        
        return {
            'input_ids':x['input_ids'].flatten(),
            'attention_mask':x['attention_mask'].flatten(),
            'labels':y['input_ids'].flatten(),
            'decoder_attention_mask':y['attention_mask'].flatten()
        };

In [ ]:
training_dataset = CustomDataset(
    path_x='/kaggle/working/rev_trainset_cmb.txt',
    path_tags = "/kaggle/working/train_tags.txt",
    path_y="/kaggle/working/spoiler_y.txt",
    tokenizer=tokenizer,
    max_seq_length=512,
    max_target_length=512,
);

validation_dataset = CustomDataset(
    path_x='/kaggle/working/rev_valset_cmb.txt',
    path_tags = "/kaggle/working/validation_tags.txt",
    path_y="/kaggle/working/spoiler_y_val.txt",
    tokenizer=tokenizer,
    max_seq_length=512,
    max_target_length=512,
);

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir='/kaggle/working/',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
);

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=training_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
);

In [ ]:
trainer.train();